In [ ]:
import pandas as pd
import numpy as np
import scipy
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scripts.python.routines.betas import betas_drop_na
from plotly.subplots import make_subplots
from scipy import stats
import pickle
import random
import plotly.express as px
import copy
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scripts.python.pheno.datasets.filter import filter_pheno
from scripts.python.pheno.datasets.features import get_column_name, get_status_dict, get_sex_dict
from scripts.python.routines.plot.scatter import add_scatter_trace
import plotly.graph_objects as go
import pathlib
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout, get_axis
from scripts.python.routines.plot.p_value import add_p_value_annotation
from statsmodels.stats.multitest import multipletests
from sklearn.metrics import mean_absolute_error
from impyute.imputation.cs import fast_knn, mean, median, random, mice, mode, em
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=False)
from functools import reduce
from scipy.stats import kruskal, mannwhitneyu
from pycombat import Combat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Trying combat

In [ ]:
dataset = "GSEUNN"
path = f"E:/YandexDisk/Work/pydnameth/datasets"
datasets_info = pd.read_excel(f"{path}/datasets.xlsx", index_col='dataset')
platform = datasets_info.loc[dataset, 'platform']
manifest = get_manifest(platform, path=path)

num_samples = 1052

df = pd.read_excel(f"{path}/{platform}/{dataset}/data/immuno/df_type(raw)_all({num_samples})_imp(fast_knn)_replace(quarter).xlsx", index_col="index")
feats = pd.read_excel(f"{path}/{platform}/{dataset}/data/immuno/immuno_markers_genes.xlsx").loc[:, 'gene'].values.tolist()

left = 0.05
right = 0.95
ranges = {}
for feat in feats:
    ranges[feat] = (df[feat].quantile(left), df[feat].quantile(right))

In [ ]:

Y = df.loc[:, feats].values

status_dummies = pd.get_dummies(df['Status'], drop_first=True)
sex_dummies = pd.get_dummies(df['Sex'], drop_first=True)
region_dummies = pd.get_dummies(df['Region'], drop_first=True)
X = pd.concat([df.loc[:,'Age'], status_dummies, sex_dummies, region_dummies], axis=1).values
b = df.loc[:, 'file'].values
combat = Combat()
Y_adjusted = combat.fit_transform(Y=Y, b=b, X=X)

scalers = {}

df.loc[:, feats] = Y_adjusted
df.index.name = 'index'
for feat in feats:
    vals = df.loc[:, feat].values
    scalers[feat] = MinMaxScaler(feature_range=ranges[feat])
    vals_scaled = scalers[feat].fit_transform(vals.reshape(-1, 1))
    df.loc[:, feat] = vals_scaled
df.to_excel(f"{path}/{platform}/{dataset}/data/immuno/df_type(minmax_left({left})_right({right})_combat)_all({num_samples})_imp(fast_knn)_replace(quarter).xlsx", index=True)
transforms = {
    "ranges": ranges,
    "scalers": scalers
}
with open(f"{path}/{platform}/{dataset}/data/immuno/transforms_minmax_left({left})_right({right})_all({num_samples})_imp(fast_knn)_replace(quarter).pickle", 'wb') as handle:
    pickle.dump(transforms, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Read the transforms

In [ ]:
with open(f"{path}/{platform}/{dataset}/data/immuno/transforms_minmax_left({left})_right({right})_all({num_samples})_imp(fast_knn)_replace(quarter).pickle", 'rb') as handle:
    transforms = pickle.load(handle)